## Step1 : Create Store Class 


In [3]:
from sequence.kernel.timeline import Timeline

class Store(object):
    def __init__(self, tl: Timeline):
        self.opening = False
        self.timeline = tl

    def open(self) -> None:
        self.opening = True

    def close(self) -> None:
        self.opening = False

## Step 2: Schedule Events on Store

In [4]:
from sequence.kernel.event import Event
from sequence.kernel.process import Process

tl = Timeline() # create timeline
tl.show_progress = False # turn of progress bar, we will address this in later tutorials.
store = Store(tl) # create store

# open store at 7:00
open_proc = Process(store, 'open', []) # Process(object, function name: string, arguments of function: List[])
open_event = Event(7, open_proc) # Event(occurring time: int, process: Process)
tl.schedule(open_event) # Timeline.schedule(Event)

## In the above code, we first create instances of Timeline and Store. Then, we create a process with the object being Store, the method of Store and a list of arguments for the method. The Event class uses open_proc process to create an event occurring at 7:00. The object Timeline then schedules the created event in its event list.

In [5]:
print(store.opening) # False

False


## The shop is always closed until the scheduled event is executed. For the event to run and the shop to open, the timeline simulator must advance to time 7.

## Step 3: Run Simulation